In [1]:
import os
import json
import sqlite3
import voyageai
import anthropic


import numpy as np
import pandas as pd

from dotenv import load_dotenv
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [3]:
class AnthropicCalls:
    def __init__(
            self,
            name="Anthropic Chat",
            api_key="",
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            temperature=0.7,
            system_prompt="",
            stream=False,
    ):
        self.name = name
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = system_prompt
        self.stream = stream
        self.history = []

        self.client = anthropic.Anthropic(
            api_key=self.api_key,
        )

    def add_message(self, role, content):
        self.history.append(
            {
                "role": role, 
                "content": content
            }
        )
    
    def clear_history(self):
        self.history.clear()

    def chat(self, message, **kwargs):
        self.add_message("user", message)
        return self.get_response(**kwargs)
        
    def get_response(self, should_print=True, **kwargs) -> str:
        params = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "messages": self.history,
            "system": self.system_prompt,
            **kwargs
        }
        assistant_response = ""
        text_response = ""

        if self.stream:
            with self.client.messages.stream(
                **params
            ) as stream:
                for text_chunk in stream.text_stream:
                    text_response += str(text_chunk)
                    if should_print:
                        print(text_chunk, end="", flush=True)
                assistant_response = stream.get_final_message()
        else:
            assistant_response = self.client.messages.create(
                **params
            )
            text_response = assistant_response.content[0].text
            if should_print:
                print(text_response, end="")

        if should_print:
            print()

        self.add_message("assistant", text_response)
        return assistant_response


In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

calls.chat(
    message="Hi!",
)


In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

calls.chat(
    message="Write a Long abstract",
)


In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

n = 20
message = ""
while message != "END": 
    message = input("User:")
    calls.chat(message=f"{message} please use {n} words or less")


In [ ]:
calls.history

Let's implement RAG!

In [9]:
class SQLiteCalls:
    def __init__(
            self,
            db_path="sqlite.db"
    ):
        self.db_path = db_path
        self.setup_database()

    def setup_database(self):
        conn = sqlite3.connect(self.db_path)

        # cursor = conn.cursor()
        # cursor.execute('DROP TABLE IF EXISTS chat_history')
        
        conn.execute("""
            CREATE TABLE IF NOT EXISTS chat_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                role TEXT,
                message TEXT,
                embedding TEXT,
                date TEXT DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        conn.close()

    def save_message(self, role: str, message: str, embedding):
        conn = sqlite3.connect(self.db_path, timeout=5)
        try:
            cursor = conn.cursor()
            embedding_str = json.dumps(embedding) if not isinstance(embedding, str) else embedding

            cursor.execute("""
                INSERT INTO chat_history (role, message, embedding)
                VALUES (?, ?, ?)
            """, (role, message, embedding_str))

            conn.commit()
        finally:
            conn.close()

    def load_chat_to_dataframe(self, role=""):
        conn = sqlite3.connect(self.db_path, timeout=5)
        try:
            if role == "user" or role == "assistant":
                df = pd.read_sql_query('''
                    SELECT role, message, embedding, date FROM chat_history
                    WHERE role = ? 
                    ORDER BY date ASC                
                ''', conn, params=(role, ))
            else:
                df = pd.read_sql_query('''
                    SELECT * FROM chat_history
                    ORDER BY date ASC                
                ''', conn)
        finally:
            conn.close()
        return df


Adding Embedder

In [10]:
class AnthropicCalls:
    def __init__(
            self,
            name="Anthropic Chat",
            api_key="",
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            temperature=0.7,
            system_prompt="",
            stream=False,
    ):
        self.name = name
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = system_prompt
        self.stream = stream
        self.history = []

        self.client = anthropic.Anthropic(
            api_key=self.api_key,
        )

        # self.embeder = voyageai.Client(
        #     api_key=self.api_key,
        # )

        self.embeder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

    def add_message(self, role, content):
        self.history.append(
            {
                "role": role, 
                "content": content
            }
        )
    
    def clear_history(self):
        self.history.clear()

    def chat(self, message, clear_after_response=False, **kwargs) -> str:
        self.add_message("user", message)
        response = self.get_response(**kwargs)
        
        if clear_after_response:
            self.clear_history()
        return response
        
    def get_response(self, should_print=True, **kwargs) -> str:
        params = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "messages": self.history,
            "system": self.system_prompt,
            **kwargs
        }
        assistant_response = ""
        text_response = ""

        if self.stream:
            with self.client.messages.stream(
                **params
            ) as stream:
                for text_chunk in stream.text_stream:
                    text_response += str(text_chunk)
                    if should_print:
                        print(text_chunk, end="", flush=True)
                assistant_response = stream.get_final_message()
        else:
            assistant_response = self.client.messages.create(
                **params
            )
            text_response = assistant_response.content[0].text
            if should_print:
                print(text_response, end="")

        if should_print:
            print()

        self.add_message("assistant", text_response)
        return assistant_response
        
    def get_embedding(self, text):
        text = text.replace("\n", " ")
        # return self.embeder.embed(
        #     texts=[text],
        #     model=model
        # ).embeddings[0]
        return self.embeder([text])[0]


In [ ]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

LLM_calls.get_embedding("When is Apple's conference call scheduled?")

In [12]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)
SQL_calls = SQLiteCalls()


def get_context(embedding, role="", n=1):
    chat_df = SQL_calls.load_chat_to_dataframe(role)
    context = find_top_n_similar(chat_df, embedding, n)
    return context


def find_top_n_similar(df, user_input_embedding, n=5):
    if df.empty or 'embedding' not in df.columns:
        print("The DataFrame is empty or missing the 'embedding' column.")
        return pd.DataFrame()
    
    df['embedding'] = df['embedding'].apply(
        lambda emb: json.loads(emb) if isinstance(emb, str) else emb
    )
    df['similarity'] = df['embedding'].apply(
        lambda emb: similarty_search(user_input_embedding, emb)
    )

    top_n_df = df.sort_values(by='similarity', ascending=False).head(n)
    # To have messages in the correct order
    top_n_df = top_n_df.sort_values(by='date', ascending=True)

    return top_n_df


def similarty_search(embedding1, embedding2):
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)

    similarity = cosine_similarity(embedding1, embedding2)

    return similarity[0][0]


def send_message(text: str, clear_after_response=False) -> str:
    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)
    SQL_calls.save_message(
        role="user",
        message=text,
        embedding=embedding
    )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else:
        context_messages = context["message"].tolist()
        print("------", "Context:", *context_messages, "------", sep="\n")
        context_message = '\n'.join(context_messages)
        combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    

    llm_response = LLM_calls.chat(combined_message, clear_after_response).content[0].text
    llm_embedding = LLM_calls.get_embedding(llm_response)
    SQL_calls.save_message(
        role="assistant",
        message=llm_response, 
        embedding=llm_embedding
    )

    return llm_response

In [13]:
def conversation():
    message = ''

    while message != "END":
        message = input("User: ")
        print("------", message, "------", sep='\n')
        if message != "END":
            response = send_message(message, clear_after_response=True)

        # LLM_calls.clear_history() # clear history of conversation

In [ ]:
conversation()

In [ ]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

get_context(
    embedding = LLM_calls.get_embedding("What is my name?"), 
    role="user", 
    n=3
) 
# ['message'] .to_list()

In [ ]:
conversation()

So it is not really convenient
(mess with questions, saving everything)

In [27]:
SQL_calls = SQLiteCalls("key_words.db")


def send_message(text: str, clear_after_response=False) -> str:
    key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=2)

    if any([ word in text.lower() for word in key_words ]): # Now we are saving only messages with key words
        SQL_calls.save_message(
            role="user",
            message=text,
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else:
        context_messages = context["message"].tolist()
        print("Context:", *context_messages, "------", sep="\n")
        context_message = '\n'.join(context_messages)
        combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

Now it's somewhat valid, but still have some issues (provides useless context when there is nothing else to give)

Let's add LLM determination if context is relevant

In [29]:
context_determinator = AnthropicCalls(
    api_key=ANTHROPIC_API_KEY, 
    max_tokens=400,
    system_prompt="You are a helpful assistant that determines if a chunk of text is relevant to a given query.\n" +
        "Respond with JSON object containing a boolean 'is_relevant' field and a 'reason' field explaining your decision"
)


def is_relevant(chunk: str, query: str):
    response = context_determinator.chat(
        f"Query: {query}\n\nChunk: {chunk}\n\nIs this chunk relevant to the query? Respond in JSON format.",
        should_print=False,
        clear_after_response=True
    )
    print("Chunk:\n", chunk)
    print("Response:\n", response.content[0].text)
    return json.loads(response.content[0].text)["is_relevant"]


def send_message(text: str, clear_after_response=False) -> str:
    key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)

    if any([ word in text.lower() for word in key_words ]):
        SQL_calls.save_message(
            role="user",
            message=text,
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else: # Now we will check if context we get is relevant to our message
        context_messages = context["message"].tolist()
        cleared_context = []
        for context_chunk in context_messages:
            if is_relevant(context_chunk, text):
                cleared_context.append(context_chunk)

        # If there are any chunks left:
        if len(cleared_context) > 0:
            print("Context:", *cleared_context, "------", sep="\n")
            context_message = '\n'.join(cleared_context)
            combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    
        else:
            combined_message = text

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

Cool! That's already much better!

Now let's head to other option

User data extractor

In [32]:
user_data_extractor = AnthropicCalls(
    api_key=ANTHROPIC_API_KEY, 
    max_tokens=1024,
    system_prompt="You are a helpful assistant that extracts chunk of user related data from given query.\n"
)

extractor_tool = {
    "name": "save_extracted_data",
    "description": "Save a given data extracted from the users query.",
    "input_schema": {
        "type": "object",
        "properties": {
            "chunk": {
                "type": "string",
                "description": "Extracted data."
            }
        },
        "required": ["chunk"]
    }
}


def extract_user_data(query: str):
    response = user_data_extractor.chat(
        f"Query: {query}\n\nExtracts user related data from this query? Use tools to save it.",
        should_print=False,
        tools = [extractor_tool],
        clear_after_response=True
    )
    if response.stop_reason == "tool_use":
        for item in response.content:
            if item.type == "tool_use":
                if item.name == "save_extracted_data":
                    print("\nTool_use: ", item)
                    print("------")
                    return item.input.get("chunk", False)
    # print("Response:\n", response)
    return False


def send_message(text: str, clear_after_response=False) -> str:
    # key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)

    extracted_data = extract_user_data(text)
    if extracted_data:
        SQL_calls.save_message(
            role="user",
            message=str(extracted_data),
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else: # Now we will check if context we get is relevant to our message
        context_messages = context["message"].tolist()
        cleared_context = []
        for context_chunk in context_messages:
            if is_relevant(context_chunk, text):
                cleared_context.append(context_chunk)

        # If there are any chunks left:
        if len(cleared_context) > 0:
            print("Context:", *cleared_context, "------", sep="\n")
            context_message = '\n'.join(cleared_context)
            combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    
        else:
            combined_message = text

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

In [26]:
mess = """
Атом
Материал из Википедии — свободной энциклопедии
Текущая версия страницы пока не проверялась опытными участниками и может значительно отличаться от версии, проверенной 3 сентября 2024 года; проверки требуют 9 правок.
У этого термина существуют и другие значения, см. Атом (значения).

Сравнительный размер атома гелия и его ядра
А́том (от др.-греч. ἄτομος «неделимый[1], неразрезаемый[2]») — частица вещества микроскопических размеров и массы, наименьшая часть химического элемента, являющаяся носителем его химических свойств[1][3].

Атомы состоят из ядра и электронов (точнее, электронного «облака»). Ядро атома состоит из протонов и нейтронов. Количество нейтронов в ядре может быть разным: от нуля до нескольких десятков. Если число электронов совпадает с числом протонов в ядре, то атом в целом оказывается электрически нейтральным. В противном случае он обладает некоторым положительным или отрицательным зарядом и называется ионом[1]. В некоторых случаях под атомами понимают только электронейтральные системы, в которых заряд ядра равен суммарному заряду электронов, тем самым противопоставляя их электрически заряженным ионам[3][4].

Ядро, несущее почти всю (более чем 99,9 %) массу атома, состоит из положительно заряженных протонов и незаряженных нейтронов, связанных между собой при помощи сильного взаимодействия. Атомы классифицируются по количеству протонов и нейтронов в ядре: число протонов Z соответствует порядковому номеру атома в периодической системе Менделеева и определяет его принадлежность к некоторому химическому элементу, а число нейтронов N — определённому изотопу этого элемента. Единственный стабильный атом, не содержащий нейтронов в ядре — лёгкий водород (протий). Число Z также определяет суммарный положительный электрический заряд (Z×e) атомного ядра и число электронов в нейтральном атоме, задающее его размер[5].

Атомы различного вида в разных количествах, связанные межатомными связями, образуют молекулы.


Краткий обзор различных семейств элементарных и составных частиц и теории, описывающие их взаимодействия. Элементарные частицы слева — фермионы, справа — бозоны. (Термины — гиперссылки на статьи Википедии)

Содержание
1	История становления понятия
2	Модели атомов
2.1	Квантово-механическая модель атома
3	Строение атома
3.1	Субатомные частицы
3.2	Электроны в атоме
4	Свойства атома
4.1	Масса
4.2	Размер
4.3	Радиоактивный распад
4.4	Магнитный момент
4.5	Энергетические уровни
4.6	Валентность
4.7	Дисперсионное притяжение
4.8	Деформационная поляризация атома
4.9	Ионизация атома
4.10	Взаимодействие атома с электроном
5	Электроотрицательность атома
6	Символизм
7	См. также
8	Примечания
9	Литература
10	Ссылки
История становления понятия
Понятие об атоме как о наименьшей неделимой части материи было впервые сформулировано древнеиндийскими и древнегреческими философами (см.: атомизм). В XVII и XVIII веках химикам удалось экспериментально подтвердить эту идею, показав, что некоторые вещества не могут быть подвергнуты дальнейшему расщеплению на составляющие элементы с помощью химических методов. Однако в конце XIX — начале XX века физиками были открыты субатомные частицы и составная структура атома, и стало ясно, что реальная частица, которой было присвоено имя атома, в действительности не является неделимой.

На международном съезде химиков в Карлсруэ (Германия) в 1860 году были приняты определения понятий молекулы и атома. Атом — наименьшая частица химического элемента, входящая в состав простых и сложных веществ.

Модели атомов
Кусочки материи. Демокрит полагал, что свойства того или иного вещества определяются формой, массой, и пр. характеристиками образующих его атомов. Так, скажем, у огня атомы остры, поэтому огонь способен обжигать, у твёрдых тел они шероховаты, поэтому накрепко сцепляются друг с другом, у воды — гладки, поэтому она способна течь. Даже душа человека, согласно Демокриту, состоит из атомов[6].
Модель атома Томсона 1904 г. (модель «Пудинг с изюмом»). Дж. Дж. Томсон предложил рассматривать атом как некоторое положительно заряженное тело с заключёнными внутри него электронами. Была окончательно опровергнута Резерфордом после проведённого им знаменитого опыта по рассеиванию альфа-частиц.
Ранняя планетарная модель атома Нагаоки. В 1904 году японский физик Хантаро Нагаока предложил модель атома, построенную по аналогии с планетой Сатурн. В этой модели вокруг маленького положительного ядра по орбитам вращались электроны, объединённые в кольца. Модель оказалась ошибочной.
Планетарная модель атома Бора-Резерфорда. В 1911 году[7] Эрнест Резерфорд, проделав ряд экспериментов, пришёл к выводу, что атом представляет собой подобие планетной системы, в которой электроны движутся по орбитам вокруг расположенного в центре атома тяжёлого положительно заряженного ядра («модель атома Резерфорда»). Однако такое описание атома вошло в противоречие с классической электродинамикой. Дело в том, что, согласно классической электродинамике, электрон при движении с центростремительным ускорением должен излучать электромагнитные волны, а, следовательно, терять энергию. Расчёты показывали, что время, за которое электрон в таком атоме упадёт на ядро, совершенно ничтожно. Для объяснения стабильности атомов Нильсу Бору пришлось ввести постулаты, которые сводились к тому, что электрон в атоме, находясь в некоторых специальных энергетических состояниях, не излучает энергию («модель атома Бора-Резерфорда»). Необходимость введения постулатов Бора была следствием осознания того, что для описания атома классическая механика неприменима. Дальнейшее изучение излучения атома привело к созданию квантовой механики, которая позволила объяснить подавляющее большинство наблюдаемых фактов.
Квантово-механическая модель атома
Современная модель атома является развитием планетарной модели Бора-Резерфорда. Согласно современной модели, ядро атома состоит из положительно заряженных протонов и не имеющих заряда нейтронов и окружено отрицательно заряженными электронами. Однако представления квантовой механики не позволяют считать, что электроны движутся вокруг ядра по сколько-нибудь определённым траекториям (неопределённость координаты электрона в атоме может быть сравнима с размерами самого атома).

Химические свойства атомов определяются конфигурацией электронной оболочки и описываются квантовой механикой. Положение атома в таблице Менделеева определяется электрическим зарядом его ядра (то есть количеством протонов), в то время как количество нейтронов принципиально не влияет на химические свойства; при этом нейтронов в ядре, как правило, больше, чем протонов (см.: атомное ядро). Если атом находится в нейтральном состоянии, то количество электронов в нём равно количеству протонов. Основная масса атома сосредоточена в ядре, а массовая доля электронов в общей массе атома незначительна (несколько сотых процента массы ядра).

Массу атома принято измерять в атомных единицах массы (дальтонах), равных 1⁄12 от массы атома стабильного изотопа углерода 12C.

Строение атома
Субатомные частицы
Основная статья: Субатомная частица
Хотя слово атом в первоначальном значении обозначало частицу, которая не делится на меньшие части, согласно научным представлениям он состоит из более мелких частиц, называемых субатомными частицами. Атом состоит из электронов, протонов, все атомы, кроме водорода-1, содержат также нейтроны.

Электрон является самой лёгкой из составляющих атом частиц с массой 9,11⋅10−31 кг, отрицательным зарядом и размером, слишком малым для измерения современными методами[8]. Эксперименты по сверхточному определению магнитного момента электрона (Нобелевская премия 1989 года) показывают, что размеры электрона не превышают 10−18 м[9][10].

Протоны обладают положительным зарядом и в 1836 раз тяжелее электрона (1,6726⋅10−27 кг). Нейтроны не обладают электрическим зарядом и в 1839 раз тяжелее электрона (1,6749⋅10−27 кг)[11].

При этом масса ядра меньше суммы масс составляющих его протонов и нейтронов из-за явления дефекта массы. Нейтроны и протоны имеют сравнимый размер, около 2,5⋅10−15 м, хотя размеры этих частиц определены плохо[12].

В стандартной модели элементарных частиц как протоны, так и нейтроны состоят из элементарных частиц, называемых кварками. Наряду с лептонами, кварки являются одной из основных составляющих материи. И первые и вторые являются фермионами. Существует шесть типов кварков, каждый из которых имеет дробный электрический заряд, равный +2⁄3 или (−1⁄3) элементарного. Протоны состоят из двух u-кварков и одного d-кварка, а нейтрон — из одного u-кварка и двух d-кварков. Это различие объясняет разницу в массах и зарядах протона и нейтрона. Кварки связаны между собой сильными ядерными взаимодействиями, которые передаются глюонами[13][14].

Электроны в атоме
Основная статья: Атомная орбиталь
При описании электронов в атоме в рамках квантовой механики обычно рассматривают распределение вероятности в 3n-мерном пространстве для системы n электронов.

Электроны в атоме притягиваются к ядру, между электронами также действует кулоновское взаимодействие. Эти же силы удерживают электроны внутри потенциального барьера, окружающего ядро. Для того чтобы электрон смог преодолеть притяжение ядра, ему необходимо получить энергию от внешнего источника. Чем ближе электрон находится к ядру, тем больше энергии для этого необходимо.

Электронам, как и другим частицам, свойственен корпускулярно-волновой дуализм. Иногда говорят, что электрон движется по орбитали, что неверно. Состояние электронов описывается волновой функцией, квадрат модуля которой характеризует плотность вероятности нахождения частиц в данной точке пространства в данный момент времени, или, в общем случае, оператором плотности. Существует дискретный набор атомных орбиталей, которым соответствуют стационарные чистые состояния электронов в атоме.

Каждой орбитали соответствует свой уровень энергии. Электрон в атоме может перейти на уровень с большей энергией при столкновении данного атома с другим атомом, электроном, ионом, или же поглотив фотон соответствующей энергии. При переходе на более низкий уровень электрон отдаёт энергию путём излучения фотона, либо путём передачи энергии другому электрону (безызлучательный переход, удары второго рода). Как и в случае поглощения, при излучательном переходе энергия фотона равна разности энергий электрона на этих уровнях (см.: постулаты Бора). Частота испускаемого излучения ν связана с энергией фотона E соотношением E = hν, где h — постоянная Планка.

Свойства атома
По определению, любые два атома с одним и тем же числом протонов в их ядрах относятся к одному химическому элементу. Атомы с одним и тем же количеством протонов, но разным количеством нейтронов называют изотопами данного элемента. Например, атомы водорода всегда содержат один протон, но существуют изотопы без нейтронов (водород-1, иногда также называемый протием — наиболее распространённая форма), с одним нейтроном (дейтерий) и двумя нейтронами (тритий)[15]. Известные элементы составляют непрерывный натуральный ряд по числу протонов в ядре, начиная с атома водорода с одним протоном и заканчивая атомом оганесона, в ядре которого 118 протонов[16] Все изотопы элементов периодической системы, начиная с номера 83 (висмут), радиоактивны[17][18].

Масса
Основная статья: Атомная масса
Поскольку наибольший вклад в массу атома вносят протоны и нейтроны, суммарное число этих частиц называют массовым числом. Массу покоя атома часто выражают в атомных единицах массы (а. е. м.), которая также называется дальтоном (Да). Эта единица определяется как 1⁄12 часть массы покоя нейтрального атома углерода-12, которая приблизительно равна 1,66⋅10−24 г.[19] Водород-1 — наилегчайший изотоп водорода и атом с наименьшей массой, имеет атомный вес около 1,007825 а. е. м.[20] Масса атома приблизительно равна произведению массового числа на атомную единицу массы[21] Самый тяжёлый стабильный изотоп — свинец-208[17] с массой 207,9766521 а. е. м.[22]

Так как массы даже самых тяжёлых атомов в обычных единицах (например, в граммах) очень малы, то в химии для измерения этих масс используют моли. В одном моле любого вещества по определению содержится одно и то же число атомов (ровно 6,022 140 76⋅1023). Это число (число Авогадро) выбрано таким образом, что если масса элемента равна 1 а. е. м., то моль атомов этого элемента будет иметь массу 1 г. Например, углерод имеет массу 12 а. е. м., поэтому 1 моль углерода весит 12 г.[19]

Размер
Основная статья: Радиус атома
Атомы не имеют отчётливо выраженной внешней границы, поэтому их размеры определяются по расстоянию между ядрами одинаковых атомов, которые образовали химическую связь (ковалентный радиус) или по расстоянию до самой дальней из стабильных орбит электронов в электронной оболочке этого атома (радиус атома). Радиус зависит от положения атома в периодической системе, вида химической связи, числа ближайших атомов (координационного числа) и квантово-механического свойства, известного как спин[23]. В периодической системе элементов размер атома увеличивается при движении сверху вниз по столбцу и уменьшается при движении по строке слева направо[24]. Соответственно, самый маленький атом — это атом гелия, имеющий радиус 32 пм, а самый большой — атом цезия (225 пм)[25]. Эти размеры в тысячи раз меньше длины волны видимого света (400—700 нм), поэтому атомы нельзя увидеть в оптический микроскоп. Однако отдельные атомы можно наблюдать с помощью сканирующего туннельного микроскопа.

Малость атомов демонстрируют следующие примеры. Человеческий волос по толщине в миллион раз больше атома углерода[26]. Одна капля воды содержит 2 секстиллиона (2⋅1021) атомов кислорода, и в два раза больше атомов водорода[27]. Один карат алмаза с массой 0,2 г состоит из 10 секстиллионов атомов углерода[28]. Если бы яблоко можно было увеличить до размеров Земли, то атомы достигли бы исходных размеров яблока[29].

Учёные из Харьковского физико-технического института представили первые в истории науки снимки атома. Для получения снимков учёные использовали электронный микроскоп, фиксирующий излучения и поля (field-emission electron microscope, FEEM). Физики последовательно разместили десятки атомов углерода в вакуумной камере и пропустили через них электрический разряд в 425 вольт. Излучение последнего атома в цепочке на фосфорный экран позволило получить изображение облака электронов вокруг ядра[30].

Радиоактивный распад
Основная статья: Радиоактивный распад

Диаграмма времени полураспада (T½) в секундах для различных изотопов с Z протонами и N нейтронами.
У каждого химического элемента есть один или более изотопов с нестабильными ядрами, которые подвержены радиоактивному распаду, в результате чего атомы испускают частицы или электромагнитное излучение. Радиоактивность возникает, когда радиус ядра больше радиуса действия сильных взаимодействий (расстояний порядка 1 фм[31]).

Существуют три основные формы радиоактивного распада[32][33]:

Альфа-распад происходит, когда ядро испускает альфа-частицу — ядро атома гелия, состоящее из двух протонов и двух нейтронов. В результате испускания этой частицы возникает элемент с меньшим на два атомным номером.
Бета-распад происходит из-за слабых взаимодействий, и в результате нейтрон распадается на протон, электрон и антинейтрино, во втором случае на протон, позитрон и нейтрино. Электрон и позитрон называют бета-частицами. Бета-распад увеличивает или уменьшает атомный номер на единицу. К бета-распаду относят и обратный процесс — электронный захват, когда один из протонов атомного ядра захватывает орбитальный электрон и превращается в нейтрон, испуская электронное нейтрино.
Гамма-излучение происходит из-за перехода ядра в состояние с более низкой энергией с испусканием электромагнитного излучения. Гамма-излучение может происходить вслед за испусканием альфа- или бета-частицы после радиоактивного распада.
Каждый радиоактивный изотоп характеризуется периодом полураспада, то есть временем, за которое распадается половина ядер образца. Это экспоненциальный распад, который вдвое уменьшает количество оставшихся ядер за каждый период полураспада. Например, по прошествии двух периодов полураспада в образце останется только 25 % ядер исходного изотопа[31].

Магнитный момент
Элементарные частицы обладают внутренним квантовомеханическим свойством, известным как спин. Оно аналогично угловому моменту объекта вращающегося вокруг собственного центра масс, хотя строго говоря, эти частицы являются точечными и нельзя говорить об их вращении. Спин измеряют в единицах приведённой планковской постоянной (
ℏ{\displaystyle \hbar }), тогда электроны, протоны и нейтроны имеют спин, равный ½ 
ℏ{\displaystyle \hbar }. В атоме электроны обращаются вокруг ядра и обладают орбитальным угловым моментом помимо спина, в то время как ядро само по себе имеет угловой момент благодаря ядерному спину[34].

Магнитное поле, создаваемое магнитным моментом атома, определяется этими различными формами углового момента, как и в классической физике вращающиеся заряженные объекты создают магнитное поле. Однако наиболее значительный вклад происходит от спина. Благодаря свойству электрона, как и всех фермионов, подчиняться правилу запрета Паули, по которому два электрона не могут находиться в одном и том же квантовом состоянии, связанные электроны спариваются друг с другом, и один из электронов находится в состоянии со спином вверх, а другой — с противоположной проекцией спина — в состоянии со спином вниз. Таким образом магнитные моменты электронов сокращаются, уменьшая полный магнитный дипольный момент системы до нуля в некоторых атомах с чётным числом электронов[35].

В ферромагнитных элементах, таких как железо, нечётное число электронов приводит к появлению неспаренного электрона и к ненулевому полному магнитному моменту. Орбитали соседних атомов перекрываются, и наименьшее энергетическое состояние достигается, когда все спины неспаренных электронов принимают одну ориентацию, процесс известный как обменное взаимодействие. Когда магнитные моменты ферромагнитных атомов выравниваются, материал может создавать измеримое макроскопическое магнитное поле. Парамагнитные материалы состоят из атомов, магнитные моменты которых разориентированы в отсутствие магнитного поля, но магнитные моменты отдельных атомов выравниваются при приложении магнитного поля[35][36].

Ядро атома тоже может обладать ненулевым полным спином. Обычно при термодинамическом равновесии спины ядер ориентированы случайным образом. Однако для некоторых элементов (таких как ксенон-129) возможно поляризовать значительную часть ядерных спинов для создания состояния с сонаправленными спинами — состояния называемого гиперполяризацией. Это состояние имеет важное прикладное значение в магнитно-резонансной томографии[37][38].

Энергетические уровни
Основная статья: Энергетический уровень
Электрон в атоме находится в связанном состоянии; находясь на возбуждённом уровне, он обладает потенциальной энергией, которая пропорциональна его расстоянию от ядра. Эта энергия обычно измеряется в электронвольтах (эВ), и максимальное её значение равно энергии, которую надо передать электрону, чтобы сделать его свободным (оторвать от атома). По мере перехода электрона (в атоме) на более низкие уровни потенциальная энергия уменьшается, но превращается не в кинетическую, а в энергию излучаемых фотонов. Согласно квантовомеханической модели атома связанный электрон может занимать только дискретный набор разрешённых энергетических уровней — состояний с определённой энергией. Наинизшее из разрешённых энергетических состояний называется основным (потенциальная энергия равна нулю — электрон глубже падать уже не может), а все остальные — возбуждёнными[39].

Для перехода электрона с одного энергетического уровня на другой нужно передать ему или отнять у него энергию. Эту энергию можно сообщить атому путём удара другой частицей либо путём поглощения или, соответственно, испускания фотона, причём энергия этого фотона равна абсолютной величине разности энергий начального и конечного уровней электрона. Частота испускаемого излучения пропорциональна энергии фотона, поэтому переходы между разными энергетическими уровнями проявляются в различных областях электромагнитного спектра[40]. Каждый химический элемент имеет уникальный спектр испускания, который зависит от заряда ядра, заполнения электронных подоболочек, взаимодействия электронов, а также других факторов[41].


Пример линейчатого спектра поглощения
Когда излучение с непрерывным спектром проходит через вещество (например, газ или плазму), некоторые фотоны поглощаются атомами или ионами, вызывая электронные переходы между энергетическим состояниями, разность энергий которых равна энергии поглощённого фотона. Затем эти возбуждённые электроны спонтанно возвращаются на уровень, лежащий ниже по шкале энергии, снова испуская фотоны. Испущенные фотоны излучаются не в том направлении, в каком падал поглощённый, а произвольно в телесном угле 4 пи стерадиан. В результате в непрерывном спектре появляются участки с очень низким уровнем излучения, то есть тёмные линии поглощения. Таким образом, вещество ведёт себя как фильтр, превращая исходный непрерывный спектр в спектр поглощения, в котором имеются серии тёмных линий и полос. При наблюдении с тех углов, куда не направлено исходное излучение, можно заметить излучение с эмиссионным спектром, испускаемое атомами. Спектроскопические измерения энергии, амплитуды и ширины спектральных линий излучения позволяют определить вид излучающего вещества и физические условия в нём[42].

Более детальный анализ спектральных линий показал, что некоторые из них обладают тонкой структурой, то есть расщеплены на несколько близких линий. В узком смысле «тонкой структурой» спектральных линий принято называть их расщепление, происходящее из-за спин-орбитального взаимодействия между спином и вращательным движением электрона[43].

Взаимодействие магнитных моментов электрона и ядра приводит к сверхтонкому расщеплению спектральных линий, которое, как правило, меньше, чем тонкое.

Если поместить атом во внешнее магнитное поле, то также можно заметить расщепление спектральных линий на две, три и более компонент — это явление называется эффектом Зеемана. Он вызван взаимодействием внешнего магнитного поля с магнитным моментом атома, при этом в зависимости от взаимной ориентации момента атома и магнитного поля энергия данного уровня может увеличиться или уменьшиться. При переходе атома из одного расщеплённого состояния в другое будет излучаться фотон с частотой, отличной от частоты фотона при таком же переходе в отсутствие магнитного поля. Если спектральная линия при помещении атома в магнитное поле расщепляется на три линии, то такой эффект Зеемана называется нормальным (простым). Гораздо чаще в слабом магнитном поле наблюдается аномальный (сложный) эффект Зеемана, когда происходит расщепление на 2, 4 или более линий (аномальный эффект происходит из-за наличия спина у электронов). При увеличении магнитного поля вид расщепления упрощается, и аномальный эффект Зеемана переходит в нормальный (эффект Пашена — Бака)[44]. Присутствие электрического поля также может вызвать сравнимый по величине сдвиг спектральных линий, вызванный изменением энергетических уровней. Это явление известно как эффект Штарка[45].

Если электрон находится в возбуждённом состоянии, то взаимодействие с фотоном определённой энергии может вызвать вынужденное излучение дополнительного фотона с такой же энергией — для этого должен существовать более низкий уровень, на который возможен переход, и разность энергий уровней должна равняться энергии фотона. При вынужденном излучении эти два фотона будут двигаться в одном направлении и иметь одинаковую фазу. Это свойство используется в лазерах, которые могут испускать когерентный пучок света в узком диапазоне частот[46].

Валентность
Основная статья: Валентность
Внешняя электронная оболочка атома, если она не полностью заполнена, называется валентной оболочкой, а электроны этой оболочки называются валентными электронами. Число валентных электронов определяет то, как атом связывается с другими атомами посредством химической связи. Путём образования химических связей атомы стремятся заполнить свои внешние валентные оболочки[47].

Чтобы показать повторяющиеся химические свойства химических элементов, их упорядочивают в виде периодической таблицы. Элементы с одинаковым числом валентных электронов формируют группу, которая изображается в таблице в виде столбца (движение по горизонтальному ряду соответствуют заполнению валентной оболочки электронами). Элементы, находящиеся в самом правом столбце таблицы, имеют полностью заполненную электронами внешнюю оболочку, поэтому они отличаются крайне низкой химической активностью и называются инертными или благородными газами[48][49].

Дисперсионное притяжение
Основная статья: Дисперсионные силы
Важным свойством атома является его склонность к дисперсионному притяжению. Происхождение дисперсионных сил было объяснено в 1930 году Ф. Лондоном. Межатомное взаимодействие возникает вследствие флуктуаций заряда в двух атомах, находящихся близко друг от друга. Поскольку электроны движутся, каждый атом обладает мгновенным дипольным моментом, отличным от нуля. Если бы флуктуации электронной плотности в двух атомах были бы несогласованными, то не было бы результирующего притяжения между атомами. Однако мгновенный диполь на одном атоме наводит противоположно направленный диполь в соседнем атоме. Эти диполи притягиваются друг к другу за счёт возникновения силы притяжения, которая называется дисперсионной силой, или силой Лондона. Энергия такого взаимодействия прямо пропорциональна квадрату электронной поляризуемости атома α и обратно пропорциональна r6, где r — расстояние между двумя атомами[50].

Деформационная поляризация атома
Основная статья: Электронная поляризуемость
Деформационная поляризация проявляется в присущей атомам способности к упругой деформации их электронных оболочек под действием электромагнитных полей. Сегодняшнее понимание явления деформационной поляризации основано на представлениях о конечной упругости электронных оболочек атомов под действием электрического поля[51]. Снятие внешнего электрического поля приводит к восстановлению электронной оболочки атома.

Деформация электронной оболочки атома приводит к смещению электронной плотности в атоме, что сопровождается образованием наведённого электрического дипольного момента μ. Дипольный момент равен произведению величины положительного заряда q на расстояние между зарядами L и направлен от отрицательного заряда к положительному μ=qL. В относительно слабых электрических полях наведённый дипольный момент пропорционален напряжённости электрического поля E. μ =αeE, где αe — электронная поляризуемость атома. Наибольшее значение электронной поляризуемости наблюдается у атомов щелочных металлов, а минимальное у атомов благородных газов.

Ионизация атома
Основные статьи: Ионизация и Энергия ионизации
При высоких значениях напряжённости приложенного электрического поля наблюдается необратимая деформация атома, сопровождающаяся отрывом электрона.

Происходит ионизация атома, атом отдаёт электрон и превращается в положительно заряженный ион — катион. Отрыв электрона от атома требует затраты энергии, называемой потенциалом ионизации или энергией ионизации.

Энергия ионизации атома сильно зависит от его электронной конфигурации. Изменение энергии отрыва первого электрона в зависимости от порядкового номера элемента приведено на рисунке.

Наименьшей энергией ионизации обладают атомы щелочных металлов, наибольшей — атомы благородных газов.

Для многоэлектронных атомов энергия ионизации I1, I2, I3… соответствует отрыву первого, второго, третьего и т. д. электронов.

Взаимодействие атома с электроном
Основная статья: Сродство к электрону
Атом	Энергия сродства
к электрону, эВ[52]
F	3,62 ± 0,09
Cl	3,82 ± 0,06
Br	3,54 ± 0,06
I	3,23 ± 0,06

Зависимость сродства к электрону атома от порядкового номера элемента
Атомы могут, в той или иной степени, присоединять добавочный электрон и превращаться в отрицательный ион — анион.

Энергетический эффект процесса присоединения к нейтральному атому (Э) принято называть энергией сродства к электрону:

Э + e- → Э-.
На рисунке представлена зависимость энергии сродства к электрону атомов от порядкового номера элемента. Наибольшим сродством к электрону обладают атомы галогенов (3—4 эВ).

Электроотрицательность атома
Основная статья: Электроотрицательность
Электроотрицательность атома (χ) — фундаментальное свойство атома смещать к себе общие электронные пары в молекуле. Способность атома данного элемента к оттягиванию на себя электронной плотности по сравнению с другими элементами соединения зависит от энергии ионизации атома и его сродства к электрону. Согласно одному из определений (по Малликену) электроотрицательность атома (χ) может быть выражена как полусумма его энергии ионизации (i) и сродства к электрону (F):

𝜒
=
1
2
(
𝑖
+
𝐹
)
{\displaystyle \chi ={\frac {1}{2}}(i+F)}
Имеется около двадцати шкал электроотрицательности атома, в основу расчёта значений которых положены различные свойства веществ. Полученные значения разных шкал отличаются, но относительное расположение элементов в ряду электроотрицательностей примерно одинаково.

Детальный поиск взаимосвязи между шкалами электроотрицательности позволил сформулировать новый подход к выбору практической шкалы электроотрицательностей атомов[53].


Символизм

Основная статья: Атом в геральдике
С момента вхождения человечества в атомную эру атом приобрёл и символический смысл. Чаще всего атом изображается в виде упрощённой модели Бора-Резерфорда. Однако встречаются и более усложнённые варианты изображения. Чаще всего изображение атома символизирует атомную энергетику («мирный атом»), ядерное оружие, ядерную физику, либо науку и научно-технический прогресс в целом.

См. также
Физика атомов и молекул
Молекула
Электрон
Протон
Нейтрон


Here is the chunk we want to situate within the whole document
Ядро, несущее почти всю (более чем 99,9 %) массу атома, состоит из положительно заряженных протонов и незаряженных нейтронов, связанных между собой при помощи сильного взаимодействия.


Please give a short succinct context to situate this chunk within overall document

"""

In [ ]:
L_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, max_tokens=2000)

L_calls.chat(mess)